In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, concatenate, Lambda, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop,SGD,Nadam, Adagrad, Adadelta
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.initializers import Constant ,Orthogonal, RandomNormal, VarianceScaling, Ones, Zeros
from tensorflow.keras.constraints import Constraint, UnitNorm
from keras.callbacks import Callback, TerminateOnNaN, ModelCheckpoint
from sksurv.metrics import concordance_index_censored as concordance
import math
import time
from sklearn.preprocessing import StandardScaler


### Data import

In [185]:
Xdf = pd.read_csv('processedExpData.csv',index_col=0)

In [3]:
ydf = pd.read_csv('yData.csv',index_col=0)

In [4]:
cancer = pd.read_csv('cancerData.csv',index_col=0)

### Model and Nested Crossval Functions

In [238]:
#This is the TFCox class

class TFCox():
    def __init__(self, seed=42,batch_norm=False,l1_ratio=1,lbda=0.0001,
                 max_it=50,learn_rate=0.001,stop_if_nan=True,stop_at_value=False, cscore_metric=False,suppress_warnings=True,verbose=0):
        
        self.max_it = max_it
        self.tnan = stop_if_nan
        self.tcscore = stop_at_value
        self.lr=learn_rate
        self.cscore=cscore_metric
        np.random.seed(seed)
        tf.random.set_seed(seed)
        
        self.l1r = l1_ratio
        self.lbda=lbda
        self.bnorm = batch_norm
        self.verbose=verbose
        if suppress_warnings == True:
            import warnings
            warnings.filterwarnings('ignore')
       
    def coxloss(self, state):
        
        def loss(y_true, y_pred):  

                return -K.mean((y_pred - K.log(tf.math.cumsum(K.exp(y_pred),reverse=True,axis=0)+0.0001))*state,axis=0)

        return loss

    def cscore_metric(self, state):
        def loss(y_true,y_pred):
            con = 0
            dis = 0
            for a in range(len(y_pred)):
                for b in range(a+1,len(y_pred)):                                       
                        if (y_pred[a]>y_pred[b])  & (y_pred[a]*state[a]!=0):
                            con+=1
                            
                        elif (y_pred[a]<y_pred[b])  & (y_pred[a]*state[a]!=0):
                            dis+=1
            return     con/(con+dis)
        return loss
 
    
    def fit(self, X,state,time):
        from tensorflow.python.framework.ops import disable_eager_execution
        disable_eager_execution()
        K.clear_session()
       
        
        
        self.time = np.array(time)  
        self.newindex = pd.DataFrame(self.time).sort_values(0).index
        self.X = (pd.DataFrame(np.array(X)).reindex(self.newindex))                      
        self.state = np.array(pd.DataFrame(np.array(state)).reindex(self.newindex))
        self.time  = np.array(pd.DataFrame(np.array(time)).reindex(self.newindex))                       
        inputsx = Input(shape=(self.X.shape[1],)) 
        state = Input(shape=(1,))
        
        if self.bnorm==True:
            out = BatchNormalization()(inputsx)
            out = Dense(1,activation='linear',
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(out)
        else:
            out = Dense(1,activation='linear',
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(inputsx)

        
        model = Model(inputs=[inputsx, state], outputs=out)
        if (self.tcscore != False) or (self.cscore==True) :
            model.compile(optimizer=Adam(self.lr) ,
                          loss=self.coxloss(state) , metrics=[self.cscore_metric(state)],
                          experimental_run_tf_function=False)
        else:
            model.compile(optimizer=Adam(self.lr) ,
                          loss=self.coxloss(state) ,
                          experimental_run_tf_function=False)
        
        self.model=model
        if self.verbose==1:
            print(self.model.summary())

        self.loss_history_ = []
        for its in range(self.max_it):
            self.temp_weights = self.model.get_weights()
           
            tr = self.model.train_on_batch([self.X, self.state],np.zeros(self.state.shape))
           
            self.loss_history_.append(tr) 
            
            if self.verbose == 1:
                if (self.tcscore != False) or (self.cscore==True) :
                    print('loss:', self.loss_history_[-1][0],' C-score: ',self.loss_history_[-1][1] )
                else:
                    print('loss:', self.loss_history_[-1] )
            
            if self.tcscore != False:
                if self.loss_history_[-1][1]>=self.tcscore:
                    print('Terminated early because concordance >=' +str(self.tcscore)+ ' as set by stop_at_value flag.')
                    break
            if (self.tcscore != False) or (self.cscore==True) :
                if (math.isnan(self.loss_history_[-1][0]) or math.isinf(self.loss_history_[-1][0])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            else:
                if (math.isnan(self.loss_history_[-1]) or math.isinf(self.loss_history_[-1])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            
        self.beta_ = self.model.get_weights()[-1]

    def predict(self,X):
        preds = self.model.predict([X,np.zeros(len(X))])

        return preds

In [245]:
#This is a function that runs stratified nested shuffle splits using the TFCox class

def nestedshuffle(lbda=[0.2], l1_r=1 ,outloop=5,inloop=5,verbose=1):
    i=0
    np.random.seed(42)
    t0=time.time()
    index_map = pd.DataFrame(range(len(Xdf)),index=Xdf.index)

    outres_train = {}
    outres = {}
    inner_best = {}
    inner_best_train={}
    inner_var = {}
    errors = {}
    preds={}
    weights={}
    timeO = {}
    
       
    
    for out in range(outloop):
        np.random.seed(out)
        t1=time.time()

        totres_train = pd.DataFrame()
        totres = pd.DataFrame()
        totres_var = pd.DataFrame()
        idx=[]
        for cnc in list(cancer):
            for st in [0,1]:

                idx.extend(index_map.loc[ np.random.choice(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)].index, 
                                        size=int(0.8*len(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)])),replace=False)][0].tolist())


        idx = np.sort(np.ravel(np.array(idx)))

        SS = StandardScaler()
        X_train = Xdf.iloc[idx,:]
        X_train  = pd.DataFrame(SS.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
        X_test = Xdf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        X_test  = pd.DataFrame(SS.transform(X_test),index=X_test.index,columns=X_test.columns)
        y_train = ydf.iloc[idx,:]
        y_test = ydf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        can_train = cancer.iloc[idx,:]
        can_test = cancer.iloc[[x for x in range(len(Xdf)) if x not in idx],:]

        
        for lb in lbda:
            
            t1=time.time()
            results = pd.DataFrame()
            trainresults = pd.DataFrame()
            index_map2 = pd.DataFrame(range(len(X_train)),index=X_train.index)
            for b in range(inloop):
                t2=time.time()
                K.clear_session()
                np.random.seed(b)

                idx2 = []
                for cnc in list(cancer):
                    for st in [0,1]:

                        idx2.extend(index_map2.loc[ np.random.choice(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)].index, 
                                                size=int(0.75*len(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)])),replace=False)][0].tolist())


                idx2 = np.sort(np.ravel(np.array(idx2)))
                SS = StandardScaler()
                X_train2 = X_train.iloc[idx2,:]

                X_train2  = pd.DataFrame(SS.fit_transform(X_train2),index=X_train2.index,columns=X_train2.columns)
                X_test2 = X_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                X_test2  = pd.DataFrame(SS.transform(X_test2),index=X_test2.index,columns=X_test2.columns)
                y_train2 = y_train.iloc[idx2,:]
                y_test2 = y_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                can_train2 = can_train.iloc[idx2,:]
                can_test2 = can_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]

                resdict = {}
                trainresdict = {}
                for a in list(cancer):

                    Xt2 = X_train2[can_train2[a]==1]
                    Xts2 = X_test2[can_test2[a]==1]




                    K.clear_session()


                    model = TFCox(lbda = lb,l1_ratio=l1_r,verbose=verbose)


                    model.fit(Xt2,y_train2['vital_status'][can_train2[a]==1],y_train2['time'][can_train2[a]==1])


                    train_pred =  model.predict(Xt2).flatten()
                    pred = model.predict(Xts2).flatten()


                    resdict[a] = pred
                    trainresdict[a] = train_pred

                res = pd.DataFrame()
                trainres = pd.DataFrame()

                for a in list(cancer):
                  #  print(trainresdict[a].shape)
                   # print(y_train2['vital_status'][can_train2[a]==1].astype('bool').shape)
                    try:

                        trainres.loc[a,0]= concordance(y_train2['vital_status'][can_train2[a]==1].astype('bool'),y_train2['time'][can_train2[a]==1],trainresdict[a])[0]
                    except:

                        trainres.loc[a,0] = 'fail'

                    try:
                        res.loc[a,0]= concordance(y_test2['vital_status'][can_test2[a]==1].astype('bool'),y_test2['time'][can_test2[a]==1],resdict[a])[0]

                    except:
                        res.loc[a,0] = 'fail'


                print(i,'/',len(lbda) *outloop*(inloop+1) )
                print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
                results[str(b)] = res[0]
                trainresults[str(b)] = trainres[0]
                i+=1

            totres_var[frozenset({'lambda':lb}.items())] = results.var(1)
            totres[frozenset({'lambda':lb}.items())] = results.mean(1)
            totres_train[frozenset({'lambda':lb}.items())] = trainresults.mean(1)


        inner_best[out] = totres
        inner_best_train[out] = totres_train
        inner_var[out] = totres_var
        outres[out] = pd.DataFrame()
        outres_train[out] = pd.DataFrame()
        timeO[out] = pd.DataFrame()
        
        
        
        for prms in list(inner_best[out]):
            K.clear_session()
            params = dict(prms)


            resdict = {}
            trainresdict = {}
            for a in list(cancer):
            
                Xt = X_train[can_train[a]==1]
                Xts = X_test[can_test[a]==1]
                
                
                t5=time.time()
             


                K.clear_session()

                model = TFCox(lbda = params['lambda'],l1_ratio=l1_r,verbose=verbose)


                model.fit(Xt,y_train['vital_status'][can_train[a]==1],y_train['time'][can_train[a]==1])


                train_pred =  model.predict(Xt).flatten()
                pred = model.predict(Xts).flatten()


                resdict[a] = pred
                trainresdict[a] = train_pred

                timeO[out].loc[a,frozenset(params.items())] = time.time()-t5
                
            res = pd.DataFrame()
            trainres = pd.DataFrame()
            for a in list(cancer):
                try:

                    trainres.loc[a,0]= concordance(y_train['vital_status'][can_train[a]==1].astype('bool'),y_train['time'][can_train[a]==1],trainresdict[a])[0]
                except:

                    trainres.loc[a,0] = 'fail'
                try:
                    res.loc[a,0]= concordance(y_test['vital_status'][can_test[a]==1].astype('bool'),y_test['time'][can_test[a]==1],resdict[a])[0]

                except:
                    res.loc[a,0] = 'fail'

            print(i,'/',len(lbda) *outloop*(inloop+1) )
            print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
            i+=1
            outres[out][frozenset(params.items())] = res[0]
            outres_train[out][frozenset(params.items())] = trainres[0]
          
    return outres , outres_train,inner_best,inner_best_train,inner_var,timeO

### Running the model

In [240]:
# This is the geometric range of lambdas
vals = [float(np.format_float_positional(1 * 0.75**i, precision=2, unique=False, fractional=False, trim='k')) for i in range(25)]


In [242]:
# Nested shuffle splits with LASSO Regularization
lassoO1,lassoO2,lassoI1,lassoI2,lassovar,lassotime= nestedshuffle(lbda=vals,l1_r=1 ,outloop=5, inloop=5,verbose=0)

0 / 750
0.5875282605489095 0.5377768317858378 0.5377601981163025
1 / 750
1.1232495824495952 1.0734981536865233 0.5357047120730082
2 / 750
1.660890797773997 1.6111393690109252 0.5376250465710958
3 / 750
2.19922643105189 2.1494750022888183 0.5383194565773011
4 / 750
2.7336099624633787 2.6838585337003074 0.5343835314114889
5 / 750
3.2735183278719586 0.5398751219113668 0.5398584882418315
6 / 750
3.811134950319926 1.0774917443593344 0.5375999569892883
7 / 750
4.351973025004069 1.6183298190434774 0.5408214569091797
8 / 750
4.890200261274973 2.156557055314382 0.5382272362709045
9 / 750
5.427762071291606 2.694118865331014 0.5375456651051839
10 / 750
5.968188834190369 0.5403935035069783 0.5403935035069783
11 / 750
6.506223094463349 1.0784277637799582 0.5380177021026611
12 / 750
7.044618848959605 1.6168235182762145 0.538379168510437
13 / 750
7.584143833319346 2.156348502635956 0.5395083626111349
14 / 750
8.121954492727916 2.6941591620445253 0.5377938946088155
15 / 750
8.657712415854137 0.5357246

1.7899560928344727
0.9003689289093018
0.822458028793335
2.207869291305542
2.216646432876587
1.2842822074890137
2.1376895904541016
1.5381898880004883
2.0798823833465576
2.0867228507995605
0.5355215072631836
1.33046293258667
0.8511173725128174
0.7796683311462402
1.1679010391235352
1.8826942443847656
1.679154634475708
2.268789052963257
125 / 750
67.86256179412206 3.310782957077026 1.1657313664754232
0.5053093433380127
1.7054784297943115
5.1697304248809814
1.2897512912750244
1.7798972129821777
0.9172542095184326
0.8212988376617432
2.24255108833313
2.3957107067108154
1.2695727348327637
2.1236202716827393
1.6060092449188232
2.110924243927002
2.092089891433716
0.5529084205627441
1.3235688209533691
0.8771867752075195
0.8009562492370605
1.1693618297576904
1.9211409091949463
1.6762325763702393
2.252358913421631
126 / 750
68.495636677742 3.9438578406969707 1.7988062500953674
0.5331192016601562
1.66453218460083
5.204949617385864
1.2826335430145264
1.8002936840057373
0.9038710594177246
0.7857134342

1.5427885055541992
2.1029274463653564
2.0929739475250244
0.5358202457427979
1.3462278842926025
0.8503549098968506
0.7916402816772461
1.161165714263916
1.888869285583496
1.6446590423583984
2.2463691234588623
142 / 750
78.57215630610784 14.020377469062804 11.875325878461203
0.5021426677703857
1.685258388519287
5.169015407562256
1.2800569534301758
1.7896409034729004
0.9026033878326416
0.8083658218383789
2.2255287170410156
2.2318410873413086
1.2607064247131348
2.137448787689209
1.583449125289917
2.2306206226348877
2.107480764389038
0.5544915199279785
1.3230769634246826
0.8948900699615479
0.8010597229003906
1.1790838241577148
1.9183323383331299
1.6858367919921875
2.235996723175049
143 / 750
79.204258898894 14.652480061848959 12.507428471247355
0.5087432861328125
1.6755998134613037
5.217921733856201
1.2957432270050049
1.7893011569976807
0.9018759727478027
0.7871420383453369
2.20857572555542
2.2221004962921143
1.2577829360961914
2.1407949924468994
1.5559232234954834
2.0972912311553955
2.08624

220 / 750
121.61095525423686 0.5419460614522298 0.5419294317563375
221 / 750
122.15702797174454 1.08801877895991 0.5460560917854309
222 / 750
122.69958547751109 1.6305762847264609 0.5425413449605306
223 / 750
123.24216212034226 2.1731529275576276 0.5425766428311666
224 / 750
123.7836213906606 2.7146121978759767 0.541442620754242
225 / 750
124.32608629465103 0.5424317876497905 0.5424150745073955
226 / 750
124.86707662741343 1.0834221204121908 0.5409742554028829
227 / 750
125.41015421549479 1.6264997084935506 0.5430775880813599
228 / 750
125.95461039543152 2.1709558884302775 0.5444395740826925
229 / 750
126.49573718706766 2.7120826800664264 0.5411105910936992
230 / 750
127.03637702862422 0.5406065861384074 0.5405899564425151
231 / 750
127.57624928951263 1.080478847026825 0.5398561000823975
232 / 750
128.11957084735235 1.6238004048665364 0.5433048327763875
233 / 750
128.6608783642451 2.1651079217592875 0.5412909110387166
234 / 750
129.20017999807993 2.7044095555941263 0.5392850120862325
2

0.558760404586792
1.349398136138916
0.899080753326416
0.799980640411377
1.199662685394287
1.9502761363983154
1.6965787410736084
2.3125555515289307
284 / 750
157.31407500108082 9.128208295504253 6.95938534339269
0.4946630001068115
1.7291569709777832
5.298948526382446
1.3232367038726807
1.8103575706481934
0.8936765193939209
0.8153793811798096
2.2537782192230225
2.2633395195007324
1.2590339183807373
2.175194025039673
1.588869571685791
2.146514654159546
2.123206615447998
0.5474364757537842
1.319300651550293
0.9034717082977295
0.8116905689239502
1.2008812427520752
1.951319932937622
1.7034680843353271
2.286973714828491
285 / 750
157.95238695144653 9.766520245869954 7.597697293758392
0.5145351886749268
1.722308874130249
5.294487237930298
1.3398029804229736
1.8316841125488281
0.9204680919647217
0.8156547546386719
2.238964557647705
2.28247332572937
1.2666025161743164
2.1722664833068848
1.6099536418914795
2.163954257965088
2.1517128944396973
0.5616674423217773
1.3550310134887695
0.88641977310180

...

531 / 750
296.43946991364163 1.0909483075141906 0.5439379175504049
532 / 750
296.9847212433815 1.6361996372540792 0.5452513297398885
533 / 750
297.53101524114606 2.1824936350186666 0.5462939977645874
534 / 750
298.07509144941963 2.7265698432922365 0.5440592964490255
535 / 750
298.6203809897105 0.5452396750450135 0.5452396750450135
536 / 750
299.1687112053235 1.0935698906580607 0.5483302156130473
537 / 750
299.7141027490298 1.6389614343643188 0.5453915437062581
538 / 750
300.25916750033696 2.1840261856714887 0.5450647513071696
539 / 750
300.80332425435387 2.728182939688365 0.5441567540168762
540 / 750
301.3494380354881 0.5460805336634318 0.5460805336634318
541 / 750
301.8947484254837 1.0913909236590067 0.54529363711675
542 / 750
302.4393484632174 1.6359909613927206 0.5445834120114644
543 / 750
302.98682875235875 2.1834712505340574 0.5474636276563009
544 / 750
303.5312255700429 2.727868068218231 0.5443802396456401
545 / 750
304.0739124298096 0.5426536162694295 0.5426536162694295
546 / 75

1.3260738849639893
1.7981367111206055
0.903956413269043
0.8181233406066895
2.27162504196167
2.302135705947876
1.2760963439941406
2.1899259090423584
1.585646390914917
2.147808790206909
2.1493442058563232
0.5582914352416992
1.3228049278259277
0.9049742221832275
0.8066072463989258
1.1772098541259766
1.9395198822021484
1.7266507148742676
2.307589292526245
586 / 750
327.5357758720716 10.41175646384557 8.238751204808553
0.5224270820617676
1.7030322551727295
5.300676107406616
1.3115696907043457
1.8165249824523926
0.9124910831451416
0.8358445167541504
2.2416746616363525
2.2822813987731934
1.271796703338623
2.1791911125183105
1.5925958156585693
2.141859292984009
2.1372976303100586
0.558140754699707
1.3414332866668701
0.8933777809143066
0.8134434223175049
1.209885835647583
1.9591233730316162
1.7071714401245117
2.306234836578369
587 / 750
328.1766287525495 11.052609344323477 8.879604085286458
0.5188641548156738
1.7279419898986816
5.309522390365601
1.3275434970855713
1.79905366897583
0.90853834152

624 / 750
349.49579730828606 2.724610527356466 0.5450656970342
625 / 750
350.0411904652913 0.5453598777453105 0.5453598777453105
626 / 750
350.5858294526736 1.0899988651275634 0.5446223656336466
627 / 750
351.13123480478924 1.6354042172431946 0.5454053521156311
628 / 750
351.677086408933 2.1812558213869733 0.5458516041437785
629 / 750
352.22211371660234 2.7262831290562946 0.5450273076693217
630 / 750
352.76616175572076 0.5440148075421651 0.5439981818199158
631 / 750
353.3123211860657 1.0901742378870647 0.5461594303448994
632 / 750
353.85902357498804 1.6368766268094381 0.5466862082481384
633 / 750
354.4035852313042 2.1814382831255594 0.5445449908574422
634 / 750
354.9476259072622 2.7254789590835573 0.5440074642499287
635 / 750
355.4911066373189 0.543447478612264 0.5434308528900147
636 / 750
356.03374205827714 1.086082899570465 0.5426192482312521
637 / 750
356.5755217432976 1.6278625845909118 0.5417796850204468
638 / 750
357.11904764175415 2.1713884830474854 0.5435258984565735
639 / 750


0.925283670425415
0.8425226211547852
2.264484167098999
2.2914645671844482
1.3042738437652588
2.1977345943450928
1.6101677417755127
2.1636481285095215
2.116896390914917
0.5494420528411865
1.3133692741394043
0.9047915935516357
0.8080594539642334
1.3220281600952148
1.9657037258148193
1.711195945739746
2.3279201984405518
728 / 750
406.4954664548238 5.293994267781575 3.113860003153483
0.5352656841278076
1.7419934272766113
5.284381151199341
1.317791223526001
1.8046576976776123
0.9181063175201416
0.8189785480499268
2.24192476272583
2.27636456489563
1.2787210941314697
2.196190357208252
1.618518352508545
2.1560661792755127
2.1509549617767334
0.5668237209320068
1.3447620868682861
0.8877596855163574
0.8152670860290527
1.1918361186981201
1.9632196426391602
1.700922966003418
2.3190979957580566
729 / 750
407.1378523071607 5.936380120118459 3.756245855490367
0.5305311679840088
1.7218856811523438
5.290099620819092
1.3193199634552002
1.8002479076385498
0.9020371437072754
0.8200578689575195
2.2579922676

2.192707061767578
1.586810827255249
2.141629934310913
2.1216464042663574
0.5543766021728516
1.3379302024841309
0.8918683528900146
0.8061394691467285
1.2011399269104004
2.0818045139312744
1.7128496170043945
2.3156869411468506
745 / 750
417.3832737763723 16.18180158933004 14.001667324701945
0.5038454532623291
1.7390196323394775
5.271883726119995
1.3180198669433594
1.7941362857818604
0.8942439556121826
0.8330905437469482
2.282043933868408
2.2632651329040527
1.2922649383544922
2.2026617527008057
1.5846707820892334
2.1575679779052734
2.1221978664398193
0.5225615501403809
1.3523001670837402
0.8754208087921143
0.8148560523986816
1.1933226585388184
1.960031509399414
1.6973004341125488
2.306788682937622
746 / 750
418.0235174338023 16.82204524676005 14.641910982131957
0.5089936256408691
1.730041742324829
5.328537702560425
1.322725772857666
1.801872730255127
0.9081182479858398
0.7943313121795654
2.260038137435913
2.246804714202881
1.2938172817230225
2.2023065090179443
1.5634489059448242
2.1694295

In [251]:
# Nested shuffle splits with elastic-net l1_ratio=0.5 Regularization
elnetO1,elnetO2,elnetI1,elnetI2,elnetvar,elnettime= nestedshuffle(lbda=vals,l1_r=0.5 ,outloop=5, inloop=5,verbose=0)

0 / 750
0.6070914308230082 0.5577080011367798 0.5576913674672445
1 / 750
1.1561887423197428 1.1068053126335144 0.5490973114967346
2 / 750
1.7067407210667929 1.6573572913805643 0.5505353649457295
3 / 750
2.2568038582801817 2.2074204285939536 0.5500462571779887
4 / 750
2.805880816777547 2.756497387091319 0.5490607857704163
5 / 750
3.3600551803906757 0.5541410128275553 0.5541244586308797
6 / 750
3.913202977180481 1.1072888096173605 0.5531477967898051
7 / 750
4.466760655244191 1.660846487681071 0.5535576780637105
8 / 750
5.022999242941538 2.217085075378418 0.556238587697347
9 / 750
5.575399859746297 2.7694856921831765 0.5524006168047587
10 / 750
6.126315081119538 0.5508819818496704 0.5508819818496704
11 / 750
6.682002850373586 1.1065697511037191 0.5556711554527283
12 / 750
7.235223710536957 1.65979061126709 0.5532208601633708
13 / 750
7.788198018074036 2.2127649188041687 0.5529576897621155
14 / 750
8.342325580120086 2.7668924808502195 0.5541109402974447
15 / 750
8.894623895486196 0.5522650

127 / 750
79.74577734073003 4.569168778260549 2.427926655610402
128 / 750
80.37650808493296 5.199899522463481 3.058657399813334
129 / 750
81.00498373905818 5.828375176588694 3.6871330539385476
130 / 750
81.63651026089987 6.45990169843038 4.318659575780233
131 / 750
82.26571482817332 7.089106265703837 4.947864143053691
132 / 750
82.8954560081164 7.718847445646921 5.577605322996775
133 / 750
83.52714120546976 8.350532643000285 6.209290520350138
134 / 750
84.15796904563904 8.981360483169556 6.840118360519409
135 / 750
84.78964000145594 9.61303143898646 7.471789316336314
136 / 750
85.4205134510994 10.243904888629913 8.102662765979767
137 / 750
86.04435575803122 10.867747195561726 8.72650507291158
138 / 750
86.67148356040319 11.494874997933705 9.353632875283559
139 / 750
87.29886323213577 12.12225466966629 9.981012547016144
140 / 750
87.92764642635981 12.75103786389033 10.609795741240184
141 / 750
88.55203520059585 13.375426638126374 11.234184515476226
142 / 750
89.1786751707395 14.00206660

252 / 750
148.68942458232243 1.6061574657758078 0.5347007632255554
253 / 750
149.22370791037878 2.140440793832143 0.5342833280563355
254 / 750
149.75420577128727 2.6709386547406515 0.5304978609085083
255 / 750
150.28659258286157 0.5323535680770874 0.5323369423548381
256 / 750
150.82434369723003 1.070104682445526 0.5377345005671184
257 / 750
151.3613363146782 1.6070972998936972 0.5369760036468506
258 / 750
151.89806823333103 2.1438292185465495 0.5367152055104574
259 / 750
152.4352034966151 2.680964481830597 0.5371185104052226
260 / 750
152.9715272943179 0.536290979385376 0.5362743655840556
261 / 750
153.50678336222967 1.0715470472971598 0.5352394461631775
262 / 750
154.04094980955125 1.6057134946187337 0.5341498255729675
263 / 750
154.57707318464915 2.141836869716644 0.5361233750979105
264 / 750
155.11105763514837 2.675821320215861 0.5339844504992167
265 / 750
155.6484974940618 0.5374070644378662 0.5373904546101888
266 / 750
156.18329791227976 1.0722074826558432 0.5348004182179769
267 /

377 / 750
218.89137947559357 2.5852548480033875 1.5112359126408894
378 / 750
220.11840090354283 3.812276275952657 1.2270048022270204
379 / 750
221.64929266770682 5.343168040116628 1.5308752338091531
380 / 750
222.2042852640152 0.5549593488375346 0.5549593488375346
381 / 750
223.09925051927567 1.449924604098002 0.8949491540590923
382 / 750
223.64255549112957 1.993229575951894 0.543304971853892
383 / 750
224.18534148931502 2.53601557413737 0.5427692850430806
384 / 750
224.72391071716945 3.0745848019917807 0.5385526061058045
385 / 750
225.70879187981288 0.9848312894503276 0.9848312894503276
386 / 750
226.5850623289744 1.8611017386118571 0.8762538393338521
387 / 750
227.4661342740059 2.7421736836433412 0.8810553193092346
388 / 750
228.00514880418777 3.2811882138252257 0.5389978686968485
389 / 750
228.54637839396796 3.8224178036053975 0.5412133812904358
390 / 750
229.30408752361933 0.7576758782068889 0.7576593081156413
391 / 750
230.3641499598821 1.8177383144696553 1.0600624362627664
392 / 

502 / 750
295.36176815032957 1.60735924243927 0.5359192411104838
503 / 750
295.8968131780624 2.1424042701721193 0.5350450277328491
504 / 750
296.4343503514926 2.679941443602244 0.5375205516815186
505 / 750
296.96914766232175 0.5347640991210938 0.5347475012143453
506 / 750
297.5051440437635 1.070760480562846 0.5359963814417521
507 / 750
298.04086396694186 1.6064804037412008 0.5357033014297485
508 / 750
298.57648513714474 2.1421015739440916 0.5356044371922811
509 / 750
299.11144120693206 2.6770576437314353 0.534939440091451
510 / 750
299.6492028991381 0.5377118190129598 0.5377118190129598
511 / 750
300.1845489144325 1.0730578343073527 0.5353293935457866
512 / 750
300.717639541626 1.6061484615008037 0.5330739418665568
513 / 750
301.25434467792513 2.142853597799937 0.536688482761383
514 / 750
301.7915454943975 2.680054414272308 0.5371846993764241
515 / 750
302.3248994867007 0.5333207686742146 0.5333040992418925
516 / 750
302.862482492129 1.070903774102529 0.5375663916269938
517 / 750
303.3

627 / 750
364.5628036459287 1.6037199219067892 0.5362003485361735
628 / 750
365.09301738739015 2.133933663368225 0.530196996529897
629 / 750
365.62284710407255 2.6637633800506593 0.5298129836718242
630 / 750
366.1586255510648 0.5357452074686686 0.5357452074686686
631 / 750
366.6879954258601 1.0651150822639466 0.5293532252311707
632 / 750
367.22077501217524 1.5978946685791016 0.5327629288037617
633 / 750
367.75238020420073 2.129499860604604 0.5315882325172424
634 / 750
368.28258711099625 2.6597067674001056 0.5301902810732524
635 / 750
368.8134125908216 0.5308088898658753 0.5307922244071961
636 / 750
369.3447991212209 1.0621954202651978 0.5313698252042135
637 / 750
369.876113140583 1.5935094396273295 0.5312973976135253
638 / 750
370.40747077465056 2.124867073694865 0.5313410679499309
639 / 750
370.9390277584394 2.6564240574836733 0.5315402110417684
640 / 750
371.4717390656471 0.532694685459137 0.5326780597368876
641 / 750
372.0039831320445 1.064938751856486 0.5322273333867391
642 / 750
3

In [243]:
#This loop finds the lambda selected by the inner loop and returns the concordance from the outer loop. 
lassores = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(lassoI1[b].index):
        
        
        lassores.loc[a,b] =  lassoO1[b][lassoI1[b].idxmax(1)[a]][a]
 

In [252]:
#The same as above but for elastic net
elnetres = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(elnetI1[b].index):
        
        
        elnetres.loc[a,b] =  elnetO1[b][elnetI1[b].idxmax(1)[a]][a]
 

In [250]:
#The overall mean outer loop results for the lasso model
lassores.mean(1)

ACC     0.875086
BLCA    0.620687
BRCA    0.716307
CESC    0.717825
COAD    0.586551
ESCA    0.566282
GBM     0.628983
HNSC    0.626966
KIRC    0.692808
KIRP    0.825241
LGG     0.851378
LIHC    0.664812
LUAD    0.611275
LUSC    0.510859
MESO    0.711633
OV      0.575065
PAAD    0.631559
READ    0.455689
SARC    0.658155
SKCM    0.606237
STAD    0.561898
UCEC    0.673257
dtype: float64

In [255]:
#The overall mean outer loop results for the lasso model
elnetres.mean(1)

ACC     0.897036
BLCA    0.623836
BRCA    0.701582
CESC    0.714878
COAD    0.557702
ESCA    0.571662
GBM     0.635486
HNSC    0.625642
KIRC    0.692397
KIRP    0.808704
LGG     0.856821
LIHC    0.703863
LUAD    0.606031
LUSC    0.509892
MESO    0.707971
OV      0.564985
PAAD    0.616179
READ    0.449186
SARC    0.657369
SKCM    0.614505
STAD    0.540743
UCEC    0.665190
dtype: float64